import and visualization of the student database from UCI repo

In [392]:
from ucimlrepo import fetch_ucirepo 
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn import model_selection

student_performance = fetch_ucirepo(id=320) 
 
X = student_performance.data.features 
y = student_performance.data.targets 

Since is a linear regresion the grades on the first 2 quarters will be our X data and the 3rd quarter will be our y data

In [391]:
x_data=y[['G1','G2']]
y_data=y['G3']
y.head()

,G1,G2,G3
0,0,11,11
1,9,11,11
2,12,13,12
3,14,14,14
4,11,13,13


Define the algorithm

In [377]:
def LinReg(x_train,y_train,x_test,y_test):
    reg=LinearRegression().fit(x_train,y_train)
    y_pred=reg.predict(x_test)
    results=r2_score(y_test,y_pred)
    return(results,reg.coef_,reg.intercept_)

Evaluation of best fit; 
100 iterations; 
Train/Test split (80/20)

In [ ]:
best_acc=0
best_coef=[]
best_inter=0
for i in range(100):
    x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.20)
    Acc,coef,inter=LinReg(x_train,y_train,x_test,y_test)
    if Acc>best_acc:
        best_acc=Acc
        best_coef=coef
        best_inter=inter



Results

In [403]:
print("Accurracy: ",best_acc*100)

Accurracy:  93.5376587306132


$Y = B_0 + B_1 X_1 + B_2 X_2$

In [402]:
print("B0: ",best_inter,"B1: ",best_coef[0],"B2: ",best_coef[1])

B0:  -0.47080503756696324 B1:  0.16851884815189203 B2:  0.8993345194651244
